# 01 Imports and setup

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import regularizers
import matplotlib.pyplot as plt
from google.colab import drive
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/Sem 2/Wprowadzenie do uczenia maszynowego/Projekt/Dane/Stress-Lysis.csv", delimiter=",")

In [4]:
df.head(5)

,Humidity,Temperature,Step count,Stress Level
0,21.33,90.33,123,1
1,21.41,90.41,93,1
2,27.12,96.12,196,2
3,27.64,96.64,177,2
4,10.87,79.87,87,0


In [5]:
df.shape

(2001, 4)

# 02 Data analysis

In [26]:
fig = make_subplots(rows=2, cols=2)

fig.add_trace(go.Histogram(x=df['Humidity'], name='Humidity'), row=1, col=1)
fig.add_trace(go.Histogram(x=df['Temperature'], name='Temperature'), row=1, col=2)
fig.add_trace(go.Histogram(x=df['Step count'], name='Step count'), row=2, col=1)
fig.add_trace(go.Histogram(x=df['Stress Level'], name='Stress Level'), row=2, col=2)

fig.update_layout(width=1600, height=1000, paper_bgcolor='LightGrey')
fig.update_traces(opacity=0.8)

fig.show()

In [7]:
df.isna().any()

Humidity        False
Temperature     False
Step count      False
Stress Level    False
dtype: bool

# 03 Preprocessing and baseline models

In [8]:
def min_max(df, *col_names):
  for col_name in col_names:
    min_val = df[col_name].min()
    max_val = df[col_name].max()
    df[col_name] = (df[col_name] - min_val) / (max_val - min_val)

In [9]:
norm_df = df
min_max(norm_df, 'Humidity', 'Temperature', 'Step count')
norm_df.head(5)

,Humidity,Temperature,Step count,Stress Level
0,0.5665,0.5665,0.615,1
1,0.5705,0.5705,0.465,1
2,0.8560,0.8560,0.980,2
3,0.8820,0.8820,0.885,2
4,0.0435,0.0435,0.435,0


In [10]:
# Baseline model

train = norm_df.iloc[:1600][:]
test = norm_df.iloc[1600:][:]

x_train = train.drop('Stress Level', axis=1)
x_test = test.drop('Stress Level', axis=1)
y_train = train['Stress Level']
y_test = test['Stress Level']

knn = KNeighborsClassifier(n_neighbors=3)
logreg = LogisticRegression()

knn.fit(x_train, y_train)
logreg.fit(x_train, y_train)
knn_pred = knn.predict(x_test)
logreg_pred = logreg.predict(x_test)

accuracy_score(knn_pred, y_test), accuracy_score(logreg_pred, y_test)

(1.0, 0.9975062344139651)

In [11]:
def to_one_hot(labels, dimension=3):
  results = np.zeros((len(labels), dimension))
  for i, label in enumerate(labels):
    results[i, label] = 1.
  return results

In [12]:
y_train = to_one_hot(y_train)
y_test = to_one_hot(y_test)

In [27]:
y_test

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

# 04 NN models

In [ ]:
# Neural Network 1

nn1 = keras.models.Sequential([
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(3, activation='sigmoid'),
])

nn1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

history1 = nn1.fit(x_train, y_train, epochs=80, batch_size=8, validation_split=0.15)

In [24]:
history1_dict = history1.history

loss_values = history1_dict['loss']
val_loss_values = history1_dict['val_loss']

acc_values = history1_dict['accuracy']
val_acc_values = history1_dict['val_accuracy']

fig = make_subplots(rows=1, cols=2)
epochs = range(1, len(loss_values)+1)

fig.add_trace(go.Scatter(x=list(epochs), y=acc_values, name='Train acc', mode='lines', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=list(epochs), y=val_acc_values, name='Val acc', mode='lines', line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=list(epochs), y=loss_values, name='Train loss', mode='lines', line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=list(epochs), y=val_loss_values, name='Val loss', mode='lines', line=dict(color='red')), row=1, col=2)

fig.update_layout(width=1600, height=1000, paper_bgcolor='LightGrey')
fig.update_layout(
    xaxis_title='Epochs',
    yaxis_title='Accuracy',
    xaxis2_title='Epochs',
    yaxis2_title='Loss'
)

fig.show()

In [25]:
nn1.evaluate(x_test, y_test)

13/13 [==============================] - 0s 3ms/step - loss: 0.0087 - accuracy: 0.9975


[0.008684653788805008, 0.9975062608718872]

In [16]:
# Neural Network 2

nn2 = keras.models.Sequential([
    keras.layers.Dense(128, kernel_regularizer=regularizers.l2(0.002), activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, kernel_regularizer=regularizers.l2(0.002), activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, kernel_regularizer=regularizers.l2(0.002), activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3, activation='sigmoid'),
])

nn2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

history2 = nn2.fit(x_train, y_train, epochs=40, batch_size=8, validation_split=0.15)

Epoch 1/40
170/170 [==============================] - 2s 6ms/step - loss: 0.6661 - accuracy: 0.6074 - val_loss: 0.4032 - val_accuracy: 0.8875
Epoch 2/40
170/170 [==============================] - 1s 4ms/step - loss: 0.3066 - accuracy: 0.8963 - val_loss: 0.2206 - val_accuracy: 0.9167
Epoch 3/40
170/170 [==============================] - 1s 5ms/step - loss: 0.2192 - accuracy: 0.9309 - val_loss: 0.2122 - val_accuracy: 0.9042
Epoch 4/40
170/170 [==============================] - 1s 4ms/step - loss: 0.1785 - accuracy: 0.9500 - val_loss: 0.1454 - val_accuracy: 0.9708
Epoch 5/40
170/170 [==============================] - 1s 4ms/step - loss: 0.1586 - accuracy: 0.9566 - val_loss: 0.1255 - val_accuracy: 0.9792
Epoch 6/40
170/170 [==============================] - 1s 4ms/step - loss: 0.1452 - accuracy: 0.9500 - val_loss: 0.1569 - val_accuracy: 0.9375
Epoch 7/40
170/170 [==============================] - 1s 3ms/step - loss: 0.1384 - accuracy: 0.9574 - val_loss: 0.1054 - val_accuracy: 0.9875
Epoch 

In [28]:
history2_dict = history2.history

loss_values = history2_dict['loss']
val_loss_values = history2_dict['val_loss']

acc_values = history2_dict['accuracy']
val_acc_values = history2_dict['val_accuracy']

fig = make_subplots(rows=1, cols=2)
epochs = range(1, len(loss_values)+1)

fig.add_trace(go.Scatter(x=list(epochs), y=acc_values, name='Train acc', mode='lines', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=list(epochs), y=val_acc_values, name='Val acc', mode='lines', line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=list(epochs), y=loss_values, name='Train loss', mode='lines', line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=list(epochs), y=val_loss_values, name='Val loss', mode='lines', line=dict(color='red')), row=1, col=2)

fig.update_layout(width=1600, height=1000, paper_bgcolor='LightGrey')
fig.update_layout(
    xaxis_title='Epochs',
    yaxis_title='Accuracy',
    xaxis2_title='Epochs',
    yaxis2_title='Loss'
)

fig.show()

In [18]:
nn2.evaluate(x_test, y_test)

13/13 [==============================] - 0s 2ms/step - loss: 0.0697 - accuracy: 0.9676


[0.06972097605466843, 0.967581033706665]

In [ ]:
# Neural Network 3

nn3 = keras.models.Sequential([
    keras.layers.Dense(3, activation='relu'),
    keras.layers.Dense(3, activation='sigmoid'),
])

nn3.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

history3 = nn3.fit(x_train, y_train, epochs=100, batch_size=8, validation_split=0.15)

In [29]:
history3_dict = history3.history

loss_values = history3_dict['loss']
val_loss_values = history3_dict['val_loss']

acc_values = history3_dict['accuracy']
val_acc_values = history3_dict['val_accuracy']

fig = make_subplots(rows=1, cols=2)
epochs = range(1, len(loss_values)+1)

fig.add_trace(go.Scatter(x=list(epochs), y=acc_values, name='Train acc', mode='lines', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=list(epochs), y=val_acc_values, name='Val acc', mode='lines', line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=list(epochs), y=loss_values, name='Train loss', mode='lines', line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=list(epochs), y=val_loss_values, name='Val loss', mode='lines', line=dict(color='red')), row=1, col=2)

fig.update_layout(width=1600, height=1000, paper_bgcolor='LightGrey')
fig.update_layout(
    xaxis_title='Epochs',
    yaxis_title='Accuracy',
    xaxis2_title='Epochs',
    yaxis2_title='Loss'
)

fig.show()

In [21]:
nn3.evaluate(x_test, y_test)

13/13 [==============================] - 0s 2ms/step - loss: 0.0611 - accuracy: 0.9975


[0.06106893718242645, 0.9975062608718872]